In [ ]:
pip install tensorflow numpy matplotlib opencv-python


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
def load_images(image_path, size=(64,64)):
    data = []
    for img in os.listdir(image_path):
        img_arr = cv2.imread(os.path.join(image_path, img))[...,::-1] # convert BGR to RGB format
        resized_arr = cv2.resize(img_arr, size) # Reshape images to preferred size
        data.append(resized_arr)
    return np.array(data) / 255.0

# Assuming you have a folder 'dataset' with all images
images = load_images('dataset')


In [ ]:
def build_generator(latent_dim):
    model = models.Sequential([
        layers.Dense(128, activation="relu", input_dim=latent_dim),
        layers.Reshape((8, 8, 2)),
        layers.UpSampling2D(),
        layers.Conv2D(64, kernel_size=3, padding="same", activation="relu"),
        layers.UpSampling2D(),
        layers.Conv2D(3, kernel_size=3, padding="same", activation="sigmoid"),
    ])
    return model

def build_discriminator(image_shape):
    model = models.Sequential([
        layers.Conv2D(64, kernel_size=3, strides=2, input_shape=image_shape, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),
        layers.Conv2D(128, kernel_size=3, strides=2, padding="same"),
        layers.LeakyReLU(0.2),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

latent_dim = 100
discriminator = build_discriminator(images.shape[1:])
generator = build_generator(latent_dim)

# GAN Model
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
discriminator.trainable = False
gan_input = layers.Input(shape=(latent_dim,))
fake_image = generator(gan_input)
gan_output = discriminator(fake_image)
gan = models.Model(gan_input, gan_output)
gan.compile(optimizer='adam', loss='binary_crossentropy')


In [ ]:
def train_gan(generator, discriminator, gan, images, latent_dim, epochs=100, batch_size=128):
    for e in range(epochs):
        for _ in range(batch_size):
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            generated_images = generator.predict(noise)
            real_images = images[np.random.randint(0, images.shape[0], batch_size)]

            X = np.concatenate([generated_images, real_images])
            y = np.zeros(2*batch_size)
            y[:batch_size] = 0.9  # Label smoothing

            d_loss = discriminator.train_on_batch(X, y)

            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            y_gen = np.ones(batch_size)
            g_loss = gan.train_on_batch(noise, y_gen)

        print(f'Epoch {e+1} / {epochs}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}')

train_gan(generator, discriminator, gan, images, latent_dim)


In [ ]:
def show_images(generator, n_images=6):
    noise = np.random.normal(0, 1, (n_images, latent_dim))
    generated_images = generator.predict(noise)
    plt.figure(figsize=(10, 8))
    for i in range(n_images):
        plt.subplot(2, 3, i+1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    plt
